In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
print("Current working directory: ", os.getcwd())

Current working directory:  /Users/nishant/Desktop/Northeastern/Fall 2021/Hitachi/cda/uda/toy


In [4]:
import sys

sys.path.append("/Users/nishant/Desktop/Northeastern/Fall 2021/Hitachi/cda/uda") # Add root directory to PATH

In [7]:
from datetime import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

from datasets import mnist, mnist_m
from models.ganin import GaninModel
import trainers.ganin as trainer
# from trainers.ganin import train_ganin, test_ganin # getting called within /toy
from utils import transform, helper


# Set random seed to ensure deterministic behavior
helper.set_random_seed(seed=123)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cpu


In [8]:
# Install wandb ()

# !pip install wandb --upgrade

In [9]:
# import wandb
# wandb.login()

In [10]:
# Hyperparameters 
config = dict(epochs=15,
              batch_size=64,
              learning_rate=2e-4,
              classes=10,
              img_size=28,
              dataset='mnist-mnist_m',
              architecture='ganin')


In [13]:
def main(hyperparameters):

    with wandb.init(project="uda-ganin-toy", config=hyperparameters, mode='disabled'):
        model = GaninModel().to(device)
        wandb.watch(model, log='all', log_freq=100)

        # transforms
        transform_m = transform.get_transform(dataset="mnist")
        transform_mm = transform.get_transform(dataset="mnist_m")

        # dataloaders
        loaders_args = dict(
            batch_size=config["batch_size"],
            shuffle=True,
            num_workers=1,
            pin_memory=True,
        )

        train_src = torch.load("../data/mnist/processed/train.pt")
        trainloader_src = mnist.fetch(data=train_src,
                                    transform=transform_m,
                                    **loaders_args)

        # fetching testloader_m for symmetry but it is not needed in the code
        test_src = torch.load("../data/mnist/processed/test.pt")
        testloader_src = mnist.fetch(data=test_src,
                                transform=transform_m,
                                **loaders_args)

        train_tgt = torch.load("../data/mnist_m/processed/train.pt")
        trainloader_tgt = mnist_m.fetch(data=train_tgt,
                                    transform=transform_mm,
                                    **loaders_args)

        test_tgt = torch.load("../data/mnist_m/processed/test.pt")
        testloader_tgt = mnist_m.fetch(data=test_tgt,
                                    transform=transform_mm,
                                    **loaders_args)
        

        criterion_l = nn.CrossEntropyLoss() # class labels 1 to N class
        criterion_d = nn.BCEWithLogitsLoss() # domain 0/1
        optimizer = optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
        
        start_time = datetime.now()
        for epoch in range(1, config["epochs"]+1):
            
            loss = trainer.train_ganin(model, epoch, config, trainloader_src, trainloader_tgt,
                        criterion_l, criterion_d, optimizer, device)
            
            wandb.log({"epoch" : epoch, "train_loss" : loss}, step=epoch)

            accuracy = trainer.test_ganin(model, testloader_tgt, device)

            wandb.log({"epoch" : epoch, "val_accuracy" : accuracy}, step=epoch)

        end_time = datetime.now()
        print(f"Train Time for {config['epochs']} epochs: {end_time - start_time}")

    return model


In [19]:
# Run

output = main(config)

alpha:  0.32151273753163445


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch: 1/15 Batch: 300/938
Total Loss: 2.279332160949707
Epoch: 1/15 Batch: 600/938
Total Loss: 1.9475303888320923
Epoch: 1/15 Batch: 900/938
Total Loss: 1.7911322116851807
Test accuracy: 0.4772093892097473


alpha:  0.5827829453479101
Epoch: 2/15 Batch: 300/938
Total Loss: 1.395620584487915
Epoch: 2/15 Batch: 600/938
Total Loss: 1.3843733072280884
Epoch: 2/15 Batch: 900/938
Total Loss: 1.378026008605957
Test accuracy: 0.6587380766868591


alpha:  0.7615941559557646
Epoch: 3/15 Batch: 300/938
Total Loss: 1.4026139974594116
Epoch: 3/15 Batch: 600/938
Total Loss: 1.4074457883834839
Epoch: 3/15 Batch: 900/938
Total Loss: 1.4053871631622314
Test accuracy: 0.7186504602432251


alpha:  0.870061661742672
Epoch: 4/15 Batch: 300/938
Total Loss: 1.3960574865341187
Epoch: 4/15 Batch: 600/938
Total Loss: 1.3944625854492188
Epoch: 4/15 Batch: 900/938
Total Loss: 1.4006659984588623
Test accuracy: 0.7809514403343201


alpha:  0.9311096086675774
Epoch: 5/15 Batch: 300/938
Total Loss: 1.431583523750305

In [22]:
transform_m = transform.get_transform(dataset="mnist")
transform_mm = transform.get_transform(dataset="mnist_m")

# dataloaders
loaders_args = dict(
    batch_size=config["batch_size"],
    shuffle=True,
    num_workers=1,
    pin_memory=True,
)

train_src = torch.load("../data/mnist/processed/train.pt")
trainloader_src = mnist.fetch(data=train_src,
                            transform=transform_m,
                            **loaders_args)

# fetching testloader_m for symmetry but it is not needed in the code
test_src = torch.load("../data/mnist/processed/test.pt")
testloader_src = mnist.fetch(data=test_src,
                        transform=transform_m,
                        **loaders_args)

train_tgt = torch.load("../data/mnist_m/processed/train.pt")
trainloader_tgt = mnist_m.fetch(data=train_tgt,
                            transform=transform_mm,
                            **loaders_args)

test_tgt = torch.load("../data/mnist_m/processed/test.pt")
testloader_tgt = mnist_m.fetch(data=test_tgt,
                            transform=transform_mm,
                            **loaders_args)

In [24]:
next(iter(testloader_src))[1].dtype

torch.int64

In [25]:
test_src[1].dtype

torch.int64